installing dependencies


In [ ]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

importing libraries

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

GPU Available: True
GPU Name: Tesla T4


Load models and tokenizer

In [ ]:
# Using GPT-2 small as an example
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU
model = model.to('cuda')

# Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print("Model loaded and moved to GPU")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded and moved to GPU


Prepare training data

In [ ]:
training_data = [
    {"input": "Hello", "output": "Hi there! How can I help you today?"},
    {"input": "What's your name?", "output": "I'm a chatbot trained to help you!"},
    {"input": "How are you?", "output": "I'm functioning well, thank you for asking!"},
    {"input": "Tell me a joke", "output": "Why don't scientists trust atoms? Because they make up everything!"},
    {"input": "What can you do?", "output": "I can chat, answer questions, and help you with various tasks."},
    {"input": "Goodbye", "output": "Goodbye! Have a great day!"},
    {"input": "What's the weather like?", "output": "I'm an AI assistant, I can't check the real weather, but I can help you with other questions!"},
    {"input": "Who created you?", "output": "I'm an AI model fine-tuned using transformers library."},
    {"input": "What time is it?", "output": "I don't have access to real-time information, but I can help you with other questions!"},
    {"input": "Thank you", "output": "You're welcome! Let me know if you need anything else."}
]

dataset = Dataset.from_dict({
    'input': [x['input'] for x in training_data],
    'output': [x['output'] for x in training_data]
})

print("Dataset size:", len(dataset))
print("\nSample entry:", dataset[0])

Dataset size: 10

Sample entry: {'input': 'Hello', 'output': 'Hi there! How can I help you today?'}


 Optimized Training Arguments for T4 GPU

In [ ]:
def tokenize_function(examples):
    # Combine input and output with a separator
    texts = [input_text + " </s> " + output_text for input_text, output_text in zip(examples['input'], examples['output'])]

    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print("Dataset tokenized successfully!")
print("Sample tokenized data:", tokenized_dataset[0])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset tokenized successfully!
Sample tokenized data: {'input': 'Hello', 'output': 'Hi there! How can I help you today?', 'input_ids': [15496, 7359, 82, 29, 15902, 612, 0, 1374, 460, 314, 1037, 345, 1909, 30, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    gradient_accumulation_steps=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    gradient_accumulation_steps=2,
    report_to="none"  # Add this line to disable wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
# 1. First, modify the tokenization function
def tokenize_function(examples):
    # Tokenize inputs and outputs separately
    inputs = tokenizer(examples['input'],
                      truncation=True,
                      padding='max_length',
                      max_length=128)

    outputs = tokenizer(examples['output'],
                       truncation=True,
                       padding='max_length',
                       max_length=128)

    # Create the labels (same as output_ids)
    inputs['labels'] = outputs['input_ids']

    return inputs

# 2. Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 3. Remove unnecessary columns
tokenized_dataset = tokenized_dataset.remove_columns(['input', 'output'])

# 4. Set format for PyTorch
tokenized_dataset.set_format('torch')

# 5. Update training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    gradient_accumulation_steps=2,
    report_to="none"
)

# 6. Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 7. Now try training
trainer.train()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=5, training_loss=9.0621826171875, metrics={'train_runtime': 8.8264, 'train_samples_per_second': 5.665, 'train_steps_per_second': 0.566, 'total_flos': 3266150400000.0, 'train_loss': 9.0621826171875, 'epoch': 5.0})

save the model

In [ ]:
# Save the model and tokenizer
output_dir = "./my_chatbot_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

Model saved to ./my_chatbot_model


In [ ]:
def generate_response(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,  # Controls randomness (0.0 = deterministic, 1.0 = very random)
        top_p=0.9,       # Nucleus sampling
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True   # Enable sampling for more diverse responses
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with various prompts
test_prompts = [
    "Hello, how are you?",
    "What can you do?",
    "Tell me a joke",
    "What's your favorite color?",
    "How does machine learning work?"
]

print("Testing the model:")
for prompt in test_prompts:
    print(f"\nPrompt: {prompt}")
    print(f"Response: {generate_response(prompt)}")

Testing the model:

Prompt: Hello, how are you?
Response: Hello, how are you?

I have a feeling that I will be there. I am so glad that I am not. I am going to be there. I am going to be there.

I want you to know that I

Prompt: What can you do?
Response: What can you do?

The most important thing is to have a job.

You can do whatever you want, and you can do whatever you want.

I want to make a good living.

I want to make

Prompt: Tell me a joke
Response: Tell me a joke about my parents. I'm not a real person.

My parents were a bit of a joke.

I'm not a real person.

My parents were a bit of a joke.

I'm

Prompt: What's your favorite color?
Response: What's your favorite color?

We've got a few more options for your next color!

We have a few more colors to share with you!

You have a favorite color?

We have a few more options for

Prompt: How does machine learning work?
Response: How does machine learning work?

Machine learning is a process of learning how to process informatio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save to Drive
drive_path = "/content/drive/My Drive/my_chatbot_model"
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)
print(f"Model saved to Google Drive at {drive_path}")

Mounted at /content/drive
Model saved to Google Drive at /content/drive/My Drive/my_chatbot_model


In [ ]:
def generate_response_with_params(prompt, temperature=0.7, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        temperature=temperature,  # Try different values (0.1 to 1.0)
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with different temperatures
test_prompt = "Tell me a story"
print("\nWith low temperature (more focused):")
print(generate_response_with_params(test_prompt, temperature=0.3))
print("\nWith high temperature (more creative):")
print(generate_response_with_params(test_prompt, temperature=0.9))


With low temperature (more focused):
Tell me a story about a guy who was a friend of mine who was a good friend of mine who was a good friend of mine. I was a good friend of mine. I was a good friend of mine. I was a good friend of

With high temperature (more creative):
Tell me a story or two, let me know!"

So there we go. I will be at this afternoon at the studio where they had to go in and get the story. I will tell my story, I will tell my story,


In [ ]:
# Better training data with more specific responses
training_data = [
    {"input": "hey! what do you do?", "output": "I'm an AI assistant trained to help you with conversations, answer questions, and provide information. How can I help you today?"},
    {"input": "what are your capabilities?", "output": "I can engage in conversations, answer questions, help with basic tasks, and provide information on various topics."},
    {"input": "who are you?", "output": "I'm an AI chatbot created to help users with various tasks and conversations."},
    {"input": "Hello", "output": "Hi there! How can I assist you today?"},
    {"input": "How are you?", "output": "I'm functioning well, thank you! How can I help you?"},
    # Add more specific examples...
]

# Convert to Dataset and tokenize again
dataset = Dataset.from_dict({
    'input': [x['input'] for x in training_data],
    'output': [x['output'] for x in training_data]
})

# Tokenize
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['input', 'output'])
tokenized_dataset.set_format('torch')

# Train again
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Retrain
trainer.train()

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=5, training_loss=9.439251708984376, metrics={'train_runtime': 3.4042, 'train_samples_per_second': 7.344, 'train_steps_per_second': 1.469, 'total_flos': 1633075200000.0, 'train_loss': 9.439251708984376, 'epoch': 5.0})

In [ ]:
# 1. Install required packages (if not already installed)
!pip install transformers torch datasets

# 2. Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

# 3. Load model and set padding token
model_name = "microsoft/DialoGPT-medium"  # Larger model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

# Add padding token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# 4. Training data
training_data = [
    {"input": "Hey! who are you?",
     "output": "I'm an AI assistant designed to help and chat with users. I aim to be helpful and friendly!"},

    {"input": "what do you do?",
     "output": "I can help with conversations, answer questions, and assist users with various tasks. What can I help you with?"},

    {"input": "are you crazy?",
     "output": "No, I'm a normal AI assistant trying to have meaningful conversations. I aim to be helpful and coherent."},

    {"input": "you're random",
     "output": "I apologize if my responses seemed random. I aim to provide clear and relevant responses. How can I help you?"},

    {"input": "what can you do?",
     "output": "I can engage in conversations, answer questions, and help with various tasks. I always try to be helpful and clear."},

    {"input": "tell me about yourself",
     "output": "I'm an AI assistant focused on helpful and coherent conversations. I can help with various tasks and questions."},

    {"input": "are you an AI?",
     "output": "Yes, I'm an AI assistant. I'm direct and honest about what I am, and I aim to be helpful in our conversations."},

    {"input": "how do you work?",
     "output": "I'm an AI language model trained to understand and respond to text. I process your input and generate helpful responses."},

    {"input": "are you helpful?",
     "output": "Yes, I strive to be helpful and provide meaningful assistance in our conversations. What can I help you with?"},

    {"input": "do you understand me?",
     "output": "Yes, I'm designed to understand and respond to your messages in a helpful and coherent way."}
]

# 5. Tokenization function
def tokenize_function(examples):
    # Combine input and output
    conversations = []
    for inp, out in zip(examples['input'], examples['output']):
        conversations.append(f"User: {inp} Assistant: {out}")

    # Tokenize
    encodings = tokenizer(
        conversations,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors="pt"
    )

    # Create labels
    encodings['labels'] = encodings['input_ids'].clone()

    return encodings

# 6. Prepare dataset
dataset = Dataset.from_dict({
    'input': [x['input'] for x in training_data],
    'output': [x['output'] for x in training_data]
})

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['input', 'output'])
tokenized_dataset.set_format('torch')

# 7. Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,  # Increased epochs
    per_device_train_batch_size=4,
    learning_rate=2e-5,   # Adjusted learning rate
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

# 8. Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 9. Train the model
print("Starting training...")
trainer.train()
print("Training completed!")

# 10. Save the model
output_dir = "./my_chatbot_final"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

# 11. Chat function
def generate_response(prompt, max_length=100):
    full_prompt = f"User: {prompt} Assistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2  # Added to prevent repetition
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(full_prompt, "").strip()
    return response

def chat():
    print("\n=== Chat with your AI Assistant ===")
    print("Type 'quit' to exit")
    print("Type 'reset' to reset the conversation")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        if user_input.lower() == 'reset':
            print("Conversation reset!")
            continue
        response = generate_response(user_input)
        print(f"AI: {response}")

# 12. Start the chat
print("\nModel is ready! Starting chat...")
chat()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Starting training...


Step,Training Loss
1,6.356600
2,6.667000
3,6.152600
4,6.493400
5,6.476600
6,4.889800
7,1.704300
8,1.557200
9,1.346500
10,1.299100


Training completed!
Model saved to ./my_chatbot_final

Model is ready! Starting chat...

=== Chat with your AI Assistant ===
Type 'quit' to exit
Type 'reset' to reset the conversation

You: Hey! what do you do?
AI: I'm an assistant to a manager.

You: OK! are you human?
AI: I am not human

You: Are you cool?
AI: I'm very cool

You: Are you helpful?
AI: Yes.

You: Are you crazy?
AI: I am not.

You: How do you work?
AI: We focus on making an effective and engaging communication with our customers.

You: Are you an angel?
AI: Yes I am an angel

You: No!!
AI: Yes

You: Ok fine. Am i an angel?
AI: Yes I'm the most angelic person you've ever met.

You: How do you help users?
AI: Help user

You: ok 
AI: I have no idea what you're talking about

You: do you like talking to me?
AI: Yes. I like talking about the world and the people and stuff. I like making things in it

You: You build stuff?
AI: I work with the people who make this stuff.

You: You an engineer?
AI: Yes, I am.

You: OK? but are 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create project directory
!mkdir -p /content/chatbot_project
%cd /content/chatbot_project

# Create necessary directories
!mkdir -p src model

/content/chatbot_project


In [ ]:
# Create requirements.txt
%%writefile requirements.txt
transformers
torch
datasets

Writing requirements.txt


In [ ]:
# Create README.md
%%writefile README.md
# AI Chatbot using DialoGPT

A fine-tuned conversational AI using DialoGPT-medium.

## Setup
1. Install requirements:
   ```bash
   pip install -r requirements.txt

Writing README.md


In [ ]:
!apt-get install -y git
!git config --global user.email "souparnachatterjee98@gmail.com"  # Replace with your email
!git config --global user.name "Souparna"        # Replace with your name

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
# Replace these with your details
username = "SouparnaChatterjee"        # Your GitHub username
token = ""             # Your GitHub token
repo_name = "chatbot-project"    # Your repository name

# Clone the repository
!git clone https://{token}@github.com/{username}/{repo_name}.git
%cd {repo_name}

# Copy your code into a file
with open('chatbot.py', 'w') as f:
    f.write('''
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Your entire working code here
# (Copy and paste your working code)
''')

# Add, commit and push
!git add .
!git commit -m "Add chatbot code"
!git push origin main

Cloning into 'chatbot-project'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'chatbot-project'
/content
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
